# Microwaves and Radar Remote Sensing - WS2b

Dr. Ing. Francescopaolo Sica

## Coverage, Resolution, Timing and Sampling

Together with the acquisition geometry, **timing** is the second fundamental definition that sets the basic properties of an MRRS system. In this workshop, we will learn to define the fundamental timing properties and new system parameters. 

To start our simulations, we first need to check if we correctly set the environment and test if all necessary packages have been installed. The following line of code will:
1) make a check about the python environment one is currently using

2) import the needed python packages (in case of errors, the package has probably not correctly been installed in the environment)

In [4]:
import sys
print(sys.executable)

/Users/alanromer/miniconda3/envs/TUB_MSSR/bin/python


In [5]:
import scipy.constants as constants
import matplotlib.pyplot as plt
import numpy as np

### From the theory

The term **timing** describes the set of temporal variables with which the system transmits the radar signals and receives the corresponding **echoes**. To be meaningful these echoes need to be coupled with the acquisition geometry, i.e., the relative position between aircraft and footprint.

The temporal dimension in the elevation and azimuth planes of the radar antenna take the name of **range** and **azimuth**, respectively. 

#### Range-time and distances

In range, the relationship between time and distance is described by

$$t_\mathrm{r} = \frac{2\cdot r_0}{\mathrm{c}_0}\quad ,$$

where

- $r_0$ is the slant range, 
- $\mathrm{c}_0$ is the velocity of propagation of the radar signals in the medium
- the factor 2 describes the way forth and back of the signals. 

The receiving window of the radar is typically called **echo window**, and is related to the difference in the travelling time for near and far range 

$$t_\mathrm{EWL} \ge \frac{2\cdot \left(r_{0, \mathrm{max}}-r_{0, \mathrm{min}}\right)}{\mathrm{c}_0}+\tau_\mathrm{pulse}\quad ,$$

where $\tau_\mathrm{pulse}$ is the **duration of the pulses** transmitted by the radar. 

The time elapsed between two transmitted pulses is called **pulse repetition interval** (PRI) and its inverse the **pulse repetition frequency** (PRF)

The PRI should be typically larger than both the echo window length and the pulse duration

$$\mathrm{PRI}=\frac{1}{\mathrm{PRF}} > t_\mathrm{EWL} + t_\mathrm{EWP}\quad . $$

The time elapsed between the transmission of the pulse and the reception of the first echoes is typically known as **echo window position**. 


#### System sensitivity

The average power is one of the main drivers of the sensitivity of the system and can be expressed as follows

$$P_\mathrm{avg} = P_\mathrm{Tx}\cdot \tau_\mathrm{pulse}\cdot \mathrm{PRF}\quad ,$$

where $P_\mathrm{Tx}$ is the **peak power** delivered by the instrument and showed in the workshop #1b. 

#### Timing for a Real Aperture Radar

In the case of a **Real Aperture Radar** (RAR), at least one echo per footprint should be acquired to grant the continuity of the image on ground. This results in

$$\mathrm{PRF}\ge\frac{v_\mathrm{g}}{r_0\cdot \theta_\mathrm{a}}\approx \frac{v_\mathrm{g}\cdot L_\mathrm{a}}{r_0\cdot \lambda}\quad .$$

The sampling of the echo window needs to fulfill the Nyquist criterium. Therefore the **range sampling frequency** (RSF) should be    

$$\mathrm{RSF}\ge B_\mathrm{r}\quad ,$$

where $B_\mathrm{r}$ is the **bandwidth** of the transmitted pulse.

#### Timing for a Synthetic Aperture Radar

In the case of a **Synthetic Aperture Radar** (SAR), the length of the antenna can be synthetically enlarged -via processing- to **twice the size of the footprint**. 

The pulse repetition frequency therefore becomes 

$$\mathrm{PRF}\ge\frac{v_\mathrm{g}}{r_0\cdot \frac{\lambda}{2\cdot r_0\cdot \theta_\mathrm{a}}}\approx \frac{2\cdot v_\mathrm{g}}{L_\mathrm{a}}\quad .$$

**Note**: The derivation is approximate -the actual equation is proportional to the spacecraft velocity rather than the ground velocity-, but functional for the sake of clarity. 

#### Range resolution

The range resolution is proportional to the signal bandwidth. In the **slant** geometry, it can be expressed as

$$\delta r = \frac{\mathrm{c}_0}{2\cdot B_\mathrm{r}}\quad ,$$

which for better interpretation can be again projected on **ground** and approximated as 

$$\delta r_\mathrm{g} = \frac{\mathrm{c}_0}{2\cdot B_\mathrm{r}\cdot \sin\theta_\mathrm{inc}}\quad .$$

**Note**: the range resolution hence varies together with the incident angle and is therefore sensitive to the local gradients of the topography.

#### Azimuth resolution

The **azimuth resolution** of a real aperture system is the footprint, i.e., 

$$\delta x_\mathrm{RAR} = r_0\cdot \frac{\lambda}{L_\mathrm{a}}\quad .$$


Assuming the results above, the azimuth resolution of a synthetic aperture radar becomes

$$\delta x_\mathrm{SAR} = r_0\cdot \frac{\lambda}{2\cdot r_0\cdot \theta_\mathrm{a}} \approx \frac{L_\mathrm{a}}{2}\quad ;$$

**Note:** the resolution in the real aperture and synthetic aperture cases have inverse relationships with the size of the antenna. Whereas in the real aperture case an improved resolution requires the growth of the antenna, in the synthetic aperture case it is the opposite, which poses a sensitivity challenge. Furthermore, in both cases the better the resolution, the higher the PRF. 




### Exercise 1

Compute the echo window length, echo window position, range sampling and pulse repetition frequency of a side-looking airborne radar geometry, assuming a real aperture system flying at 100 m/s, operated at Ku-band with 100 MHz and a 3 m antenna. Discuss the results.

In [6]:
# Define a function that computes the range distance r0 and the incidence angle given the acquisition geometry
# inputs: rE   - Earth radius
#         h    - platform altitude
#         look - look angle

def compute_r0_thetainc(rE, h, theta_look):
    b    = (rE + h) * 2 * np.cos(theta_look)
    c    = -(rE**2 - (rE + h)**2)
    r0   = (b - np.sqrt(b**2 - 4 * c)) * 0.5
    theta_inc   = np.arccos(((rE + h)**2 - rE**2 - r0**2)/(2 * r0 * rE))
    return r0, theta_inc

In [8]:
hAir = 3.e3
rE   = 6371.e3  # Earth radius
c0 = constants.c
Br = 100*1e6

N    = 1024
theta_look = np.arange(20,45,(45-20)/N) * np.pi / 180.

# **your code here**
r0, theta_inc = compute_r0_thetainc(rE,hAir,theta_look)

t_pulse = 1/100 * 1e-6
print('t_pulse:')
print(t_pulse)

# **your code here**
t_EWL = 2*(np.max(r0)-np.min(r0))/c0 +t_pulse 
print('t_EWL:')
print(t_EWL)

# **your code here**
t_EWP = 2*(np.min(r0))/c0
print('t_EWP:')
print(t_EWP)

RSF=Br
print('RSF should be bigger or equal to:')
print(RSF)

PRF= t_EWL+ t_EWP
print('PRF should be smaller or equal to:')
print(PRF)

t_pulse:
1e-08
t_EWL:
7.009498085197951e-06
t_EWP:
2.1298954083482417e-05
RSF should be bigger then:
100000000.0
PRF should be bigger then:
2.8308452168680368e-05


### Answer 1

** **your answer here** **
not sure what to discuss... the numbers seem small?

### Exercise 2

Compute the echo window length, echo window position, and PRF of an X-band spaceborne SAR system flying at 514 km with a 4.7 m antenna and illuminating a swath of 30 km around 35 deg incidence. Discuss the results.  

In [25]:
hAir = 514.e3
rE   = 6371.e3  # Earth radius
c0 = constants.c
Br = 100*1e6

N    = 1024
theta_look = np.arange(33,35,(35-33)/N) * np.pi / 180.

# **your code here**
r0, theta_inc = compute_r0_thetainc(rE,hAir,theta_look)

# **your code here**
# compute swath with precise formula
swath_spc = hAir*(np.tan(np.max(theta_inc))-np.tan(np.min(theta_inc)))
print('swath:')
print(swath_spc)

t_pulse = 1/Br
print('t_pulse:')
print(t_pulse)

# **your code here**
t_EWL = 2*(np.max(r0)-np.min(r0))/c0 +t_pulse 
print('t_EWL:')
print(t_EWL)

# **your code here**
t_EWP = 2*(np.min(r0))/c0
print('t_EWP:')
print(t_EWP)

PRF= t_EWL+ t_EWP
print('PRF should be smaller or equal to:')
print(PRF)


swath:
31763.060078877206
t_pulse:
1e-08
t_EWL:
0.00011154878256411027
t_EWP:
0.0041607426231105825
PRF should be smaller or equal to:
0.004272291405674693


### Answer 2

** **your answer here** **
First loook all numbers become several magnitudes bigger. 
t_EWP: became bigger since the distance between satalite and earth is a lot bigger then plane and earth resulting in a greater amount of time need for the signal to make the trip twice since we cannot increase the signals speed. 
t_EWL: the explanation for this increase isn't as intuitive as for t_EWP, since it measures the time of first and last signal reaceived from a single pulse. It is probably the easiest to explain via a triangle between satelite, first echo position and last. The time measured is the time needed for difference between the distance r1(satelite to first echo position) and r2(satelite to last echo position). We can use the cosine rule to determina the distance of r2 if r1 and r3 is known(first to last echo position) and inc1 angle (incident angle of first echo position). 
 
$r2^2= r1^2+r3^2-2*r1*r3*cos(inc1)$

since inc1 is always greater than 90 this results in cos being negative and r2 always being bigger the r1 by the swath and $2*r1*r3*cos(inc1)$ meaning the sice increase is dependent on the distane to earth. leads to larger disparity between r1 and r2 which leads to a longer time needed. P.S. this disparity between r1 and r2 only increases if we add the curviture of the earth. 
PWF: is just t_EWP + t_EWL

### Exercise 3

Compute the range resolution in slant and on ground for the airborne system described in exercise 1.


In [23]:
import numpy as np                                                           

hAir = 3.e3
rE   = 6371.e3  # Earth radius
c0 = constants.c
Br = 100*1e6
N    = 1024
theta_look = np.arange(20,60,(60-20)/N) * np.pi / 180.

# **your code here**
r0, theta_inc = compute_r0_thetainc(rE,hAir,theta_look)

# **your code here**

print('slant-range resolution:')
print(c0/(2*Br))
# **your code here**
print('angles:')
print(theta_inc)
print('ground-range resolution:')
print(c0/(2*Br*np.sin(np.array(theta_inc))))


slant-range resolution:
1.49896229
angles:
[0.34923724 0.34991938 0.35060151 ... 1.04596457 1.04664762 1.04733067]
ground-range resolution:
[4.38060978 4.37242062 4.36426404 ... 1.73208689 1.73140255 1.73071956]


### Answer 3

** **your answer here** **
Based on the lecture with inceasing angle the ground resolution should worsen however for whatever reason the resolution is becoming better 

### Exercise 4

For the spaceborne system in exercise 2, compute the PRF, the azimuth resolution and discuss the results. 

In [24]:
hAir = 514.e3
rE   = 6371.e3  # Earth radius
c0 = constants.c
L = 4.7
Br = 100 *1e3

N    = 1024
theta_look = np.arange(34,36,(36-34)/N) * np.pi / 180.

# **your code here**
r0, theta_inc = compute_r0_thetainc(rE,hAir,theta_look)

# compute swath with precise formula
swath_spc = hAir*(np.tan(np.max(theta_inc))-np.tan(np.min(theta_inc)))
print('swath:')
print(swath_spc)

t_pulse = 1/Br
print('t_pulse:')
print(t_pulse)

# **your code here**
t_EWL = 2*(np.max(r0)-np.min(r0))/c0 +t_pulse  
print('t_EWL:')
print(t_EWL)

# **your code here**
t_EWP = 2*(np.min(r0))/c0
print('t_EWP:')
print(t_EWP)

# **your code here**
PRF = t_EWL+ t_EWP
print('PRF:')
print(PRF)

az=L/2
print('azimoth resolution:')
print(az)



swath:
32851.12710661093
t_pulse:
1e-05
t_EWL:
0.0001277444338338354
t_EWP:
0.004215062359768221
PRF:
0.004342806793602056
azimoth resolution:
2.35


### Answer 4

** **your answer here** ** we arent given a wavelength or the frequenzy of the signal therefore dwe use the approximation
it's nice that it is closed to the range resolution of the airplane. shows that the resolution is independent from range. ? dont know what to dicus for a single number 


### Acknowledgements 

- Dr. Ing. Marc Rodriguez-Cassola 

- Prof. Andreas Reigber